<a href="https://colab.research.google.com/github/santiagorg2401/pytorch_course_exercises/blob/main/05_pytorch_going_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Going Modular
Notebook -> Python scripts

In [9]:
!mkdir going_modular

mkdir: cannot create directory ‘going_modular’: File exists


## Get data

In [10]:
%%writefile get_data.py
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Overwriting get_data.py


In [11]:
!python get_data.py

data/pizza_steak_sushi directory exists.
Unzipping pizza, steak, sushi data...


## Create Datasets and DataLoaders

In [12]:
%%writefile going_modular/data_setup.py
"""
Handles functionality for handling PyTorch DataLoaders
"""

import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """
  Creates Training and testing DataLoaders
  Args:
    train_dir: Path to training directory,
    test_dir: Path to testing directory,
    transform: torchvision transforms to perform on data,
    batch_size: Batch size,
    num_workers: Number of workers per dataloader

  Returns:
    (train_dataloader, test_dataloader, class_names)
  """
  # use ImageFolder to create datasets
  train_data = datasets.ImageFolder(train_dir, transform)
  test_data = datasets.ImageFolder(test_dir, transform)

  # Get class names
  class_names = train_data.classes

  # Turn datasets into DataLoaders
  train_dataloader = DataLoader(train_data, batch_size, True,
                                num_workers=num_workers, pin_memory=True)
  test_dataloader = DataLoader(test_data, batch_size, False,
                               num_workers=num_workers, pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


In [13]:
from pathlib import Path
from torchvision import transforms
from going_modular import data_setup

BATCH_SIZE = 32

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Setup train and testing paths
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                               test_dir,
                                                                               data_transform,
                                                                               BATCH_SIZE)

## Create TinyVGG model

In [14]:
%%writefile going_modular/model.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Writing going_modular/model.py


In [15]:
import torch
from going_modular import model

device = "cuda" if torch.cuda.is_available() else "cpu"

model_1 = model.TinyVGG(3, 10, len(class_names)).to(device)
model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

## Engine

In [16]:
%%writefile going_modular/engine.py
import torch

from torch import nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from typing import Tuple, List, Dict
from timeit import default_timer as timer

def train_step(model: nn.Module,
               dataloader: DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device):

  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class==y).sum().item()/len(y_pred)

  # Compute average loss and accuracy per batch
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

def test_step(model: nn.Module,
              dataloader: DataLoader,
              loss_fn: nn.Module,
              device):
  model.eval()
  test_loss, test_acc = 0, 0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      test_loss += loss
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1),dim=-1)
      test_acc += (y_pred_class==y).sum().item()/len(y_pred)

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  return test_loss, test_acc

def train(model: nn.Module,
          epochs: int,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          train_dataloader: DataLoader,
          test_dataloader: DataLoader,
          device):

  results = {"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[],
             "train_time":None}
  t0 = timer()
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model,
                                       train_dataloader,
                                       loss_fn,
                                       optimizer,
                                       device)
    test_loss, test_acc = test_step(model,
                                    test_dataloader,
                                    loss_fn,
                                    device)
    # Update results per epoch
    results["train_loss"].append(train_loss.to("cpu").detach().numpy())
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss.to("cpu").detach().numpy())
    results["test_acc"].append(test_acc)

    log = f"Epoch: {epoch} | " \
          f"Train loss: {train_loss:.4f} | Train acc: {train_acc*100:.2f}% " \
          f"Test loss: {test_loss:.4f} | Test acc: {test_acc*100:.2f}%"
    print(log)

  t1 = timer()
  results["train_time"] = t1 - t0
  print(f"Training time: {t1 - t0:.3f} seconds")

  return results

Writing going_modular/engine.py


## Utilities

In [17]:
%%writefile going_modular/utils.py
import torch
import matplotlib.pyplot as plt

from pathlib import Path

def plot_loss_curves(results):
  epochs = range(len(results["train_loss"]))
  # Loss
  plt.figure(figsize=(15,7))
  plt.subplot(1, 2, 1)
  plt.plot(epochs, results["train_loss"], label="Train")
  plt.plot(epochs, results["test_loss"], label="Test")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  # Accuracy
  plt.subplot(1, 2, 2)
  plt.plot(epochs, results["train_acc"], label="Train")
  plt.plot(epochs, results["test_acc"], label="Test")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend()

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


## Train

In [18]:
%%writefile going_modular/train.py
import os
import torch
import argparse
from torchvision import transforms
import data_setup, engine, model, utils

# Set Hyperparameters
parser = argparse.ArgumentParser(description="Train a TinyVGG model on PyTorch")
parser.add_argument("--num_epochs", type=int, help="Number of epochs")
parser.add_argument("--batch_size", type=int, help="Number of samples per batch")
parser.add_argument("--hidden_units", type=int, help="TinyVGG hidden units")
parser.add_argument("--lr", type=float, help="Learning rate")
parser.add_argument("--plot", type=bool, help="Plot loss curves")
parser.add_argument("--model_name", help="Model name, include .pt ot .pth")
args = parser.parse_args()

NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
IMG_SIZE = (64, 64)
LEARNING_RATE = args.lr
PLOT = args.plot
MODEL_NAME = args.model_name

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()
])

# Create DataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                               test_dir,
                                                                               data_transform,
                                                                               BATCH_SIZE)

# Create model
model = model.TinyVGG(3, HIDDEN_UNITS, len(class_names)).to(device)

# Train
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), LEARNING_RATE)
results = engine.train(model, NUM_EPOCHS, loss_fn, optimizer,
                       train_dataloader, test_dataloader, device)

# Use utils
if PLOT:
  utils.plot_loss_curves(results)
utils.save_model(model, "models", MODEL_NAME)

Writing going_modular/train.py


In [19]:
!python going_modular/train.py --num_epochs 5 --batch_size 32 --hidden_units 10 --lr 0.0001 --plot True --model_name "model_0.pth"

  0% 0/5 [00:00<?, ?it/s]Epoch: 0 | Train loss: 1.0994 | Train acc: 28.12% Test loss: 1.0948 | Test acc: 54.17%
 20% 1/5 [00:02<00:09,  2.33s/it]Epoch: 1 | Train loss: 1.0978 | Train acc: 44.92% Test loss: 1.0977 | Test acc: 29.07%
 40% 2/5 [00:03<00:04,  1.65s/it]Epoch: 2 | Train loss: 1.1001 | Train acc: 36.33% Test loss: 1.0972 | Test acc: 40.25%
 60% 3/5 [00:04<00:03,  1.51s/it]Epoch: 3 | Train loss: 1.0973 | Train acc: 51.17% Test loss: 1.0977 | Test acc: 40.25%
 80% 4/5 [00:05<00:01,  1.32s/it]Epoch: 4 | Train loss: 1.0981 | Train acc: 32.42% Test loss: 1.0998 | Test acc: 17.71%
100% 5/5 [00:06<00:00,  1.34s/it]
Training time: 6.715 seconds
[INFO] Saving model to: models/model_0.pth


## Predict

In [20]:
%%writefile predict.py
import torch
import torchvision
import argparse

from going_modular import model

# Creating a parser
parser = argparse.ArgumentParser()

# Get hyperparameters
parser.add_argument("--image",
                    help="target image filepath to predict on")

parser.add_argument("--model_path",
                    default="models/model_0.pth",
                    type=str,
                    help="target model to use for prediction filepath")
parser.add_argument("--hidden_units", type=int, help="TinyVGG hidden units")

args = parser.parse_args()
HIDDEN_UNITS = args.hidden_units

# Setup class names
class_names = ["pizza", "steak", "sushi"]

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get the image path
IMG_PATH = args.image
print(f"[INFO] Predicting on {IMG_PATH}")

# Function to load in the model
def load_model(filepath=args.model_path):
  # Need to use same hyperparameters as saved model
  model_ = model.TinyVGG(input_shape=3,
                                hidden_units=HIDDEN_UNITS,
                                output_shape=len(class_names)).to(device)

  print(f"[INFO] Loading in model from: {filepath}")
  # Load in the saved model state dictionary from file
  model_.load_state_dict(torch.load(filepath))

  return model_

# Function to load in model + predict on select image
def predict_on_image(image_path=IMG_PATH, filepath=args.model_path):
  # Load the model
  model_ = load_model(filepath)

  # Load in the image and turn it into torch.float32 (same type as model)
  image = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)

  # Preprocess the image to get it between 0 and 1
  image = image / 255.

  # Resize the image to be the same size as the model
  transform = torchvision.transforms.Resize(size=(64, 64))
  image = transform(image)

  # Predict on image
  model_.eval()
  with torch.inference_mode():
    # Put image to target device
    image = image.to(device)

    # Get pred logits
    pred_logits = model_(image.unsqueeze(dim=0)) # make sure image has batch dimension (shape: [batch_size, height, width, color_channels])

    # Get pred probs
    pred_prob = torch.softmax(pred_logits, dim=1)

    # Get pred labels
    pred_label = torch.argmax(pred_prob, dim=1)
    pred_label_class = class_names[pred_label]

  print(f"[INFO] Pred class: {pred_label_class}, Pred prob: {pred_prob.max():.3f}")

if __name__ == "__main__":
  predict_on_image()

Writing predict.py


In [21]:
!python predict.py --hidden_units 10 --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicting on data/pizza_steak_sushi/test/sushi/175783.jpg
[INFO] Loading in model from: models/model_0.pth
[INFO] Pred class: pizza, Pred prob: 0.334
